In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/title-generation/sample_submission.csv
/kaggle/input/title-generation/vocs.pkl
/kaggle/input/title-generation/train.csv
/kaggle/input/title-generation/test.csv


## Clean train dataset

In [2]:
# pd.set_option('display.max_colwidth', -1)
# def la(df: pd.DataFrame):
#     left_aligned_df = df.style.set_properties(**{'text-align': 'left'})
#     left_aligned_df = left_aligned_df.set_table_styles(
#         [dict(selector='th', props=[('text-align', 'left')])]
#     )
#     return left_aligned_df

In [3]:
train = pd.read_csv("/kaggle/input/title-generation/train.csv")
train['A_len_ch'] = train.abstract.str.len()
train['T_len_ch'] = train.title.str.len()
train['A_len_w'] = train.abstract.str.split().map(len)
train['T_len_w'] = train.title.str.split().map(len)

# Оставляем только примеры, где количество символов/слов в абстракте больше или равно количеству символов/слов в названии
train = train[train.A_len_ch >= train.T_len_ch]
train = train[train.A_len_w >= train.T_len_w]

# удаляем все начинающееся с comment/rejoinder/discussion of
for to_delete_startswith in ["comment", "rejoinder", "discussion of"]:
    train = train[~train.abstract.str.startswith(to_delete_startswith)]
    train = train[~train.title.str.startswith(to_delete_startswith)]

# удаляем короткие абстракты с ключевым словом withdrawn, а так же любые с no abstract
train = train[~(train.abstract.str.contains("withdrawn", case=False) & (train.A_len_w < 20))]
train = train[~train.abstract.str.contains("no abstract", case=False)]

# удаляем полные дубликаты
train = pd.DataFrame.drop_duplicates(train)

# оставляем из оставшихся только те дубликаты по abstract, которые имеют меньший по длине title
train = train[~train.sort_values(by=["T_len_ch"]).abstract.duplicated(keep="first")]

train.to_csv("run_train.csv", index=False)

del train

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:24: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


## Fine tuning model

In [4]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [5]:
!pip install rouge_score
!pip install wandb --upgrade

!git clone https://github.com/huggingface/transformers.git
# Заменяем файл run_summarization.py на версию, которая совместима с установленной на каггле версией transformers
!curl https://raw.githubusercontent.com/huggingface/transformers/79d28e80b6f5a37c3f6bacf3fd708963c58b68fb/examples/pytorch/summarization/run_summarization.py \
    -o transformers/examples/pytorch/summarization/run_summarization.py

!ls ./transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 20.6 MB/s eta 0:00:00
  Attempting uninstall: wandb
    Found existing installation: wandb 0.12.16
    Uninstalling wandb-0.12.16:
      Successfully uninstalled wandb-0.12.16
Cloning into 'transformers'...
remote: Enumerating objects: 100556, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 100556 (delta 6), reused 13 (delta 3), pack-reused 100535
Receiving objects: 100% (100556/100556), 94.01 MiB | 27.65 MiB/s, done.
Resolving deltas: 100% (74188/74188), done.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 30199  100 30199    0     0   201k      0 --:--:-- --:--:-- --:--:--  201k
CITATION.cff	    README.md	       examples        setup.py
CODE_OF_CONDUCT.md  README_ko.md       hubconf.py      src
CONTRIBUTING.md     README_zh-hans.md  model_cards     

In [6]:
# pd.read_csv("/kaggle/input/title-generation/train.csv")[:500].to_csv("run_train.csv", index=False)
# pd.read_csv("/kaggle/input/title-generation/train.csv")[1000:500].to_csv("run_test.csv", index=False)

In [7]:
import wandb
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()

# I have saved my API token with "wandb_api" as Label. 
# If you use some other Label make sure to change the same below. 
wandb_api = user_secrets.get_secret("wandb_api") 

wandb.login(key=wandb_api)

# wandb.init(project="Stepik-NN-and-NLP 7.Final ATG fine-tuning")

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [8]:
!python transformers/examples/pytorch/summarization/run_summarization.py \
    --model_name_or_path facebook/bart-large-cnn \
    --do_train \
    --train_file "/kaggle/working/run_train.csv" \
    --text_column "abstract" \
    --summary_column "title" \
    --output_dir /kaggle/working/tst-summarization \
    --overwrite_output_dir \
    --per_device_train_batch_size=4 \
    --per_device_eval_batch_size=4 \
    --predict_with_generate \
    --save_total_limit=2
#     --source_prefix "summarize: " \
#    --load_best_model_at_end=true

Extracting data files: 100%|████████████████████| 1/1 [00:00<00:00, 1073.81it/s]
Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-e03253f123116e83/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.
100%|████████████████████████████████████████████| 1/1 [00:00<00:00, 291.64it/s]
[INFO|hub.py:583] 2022-07-12 12:56:37,300 >> https://huggingface.co/facebook/bart-large-cnn/resolve/main/config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpdord2me8
Downloading: 100%|█████████████████████████| 1.55k/1.55k [00:00<00:00, 1.35MB/s]
[INFO|hub.py:587] 2022-07-12 12:56:37,404 >> storing https://huggingface.co/facebook/bart-large-cnn/resolve/main/config.json in cache at /root/.cache/huggingface/transformers/199ab6c0f28e763098fd3ea09fd68a0928bb297d0f76b9f3375e8a1d652748f9.930264180d256e6fe8e4ba6a728dd80e969493c23d4caa0a6f943614c52d34ab
[INFO|h

In [9]:
# !python transformers/examples/pytorch/summarization/run_summarization.py \
#     --model_name_or_path t5-small \
#     --do_train \
#     --do_eval \
#     --train_file "/kaggle/working/run_train.csv" \
#     --validation_file "/kaggle/working/run_test.csv" \
#     --text_column "abstract" \
#     --summary_column "title" \
#     --source_prefix "summarize: " \
#     --output_dir /kaggle/working/tst-summarization \
#     --overwrite_output_dir \
#     --per_device_train_batch_size=30 \
#     --per_device_eval_batch_size=30 \
#     --predict_with_generate

In [10]:
!ls -la /kaggle/working/tst-summarization

total 1590800
drwxr-xr-x 5 root root       4096 Jul 12 21:47 .
drwxr-xr-x 5 root root       4096 Jul 12 12:58 ..
-rw-r--r-- 1 root root       1030 Jul 12 21:47 README.md
-rw-r--r-- 1 root root        196 Jul 12 21:47 all_results.json
drwxr-xr-x 2 root root       4096 Jul 12 21:42 checkpoint-78000
drwxr-xr-x 2 root root       4096 Jul 12 21:45 checkpoint-78500
-rw-r--r-- 1 root root       1655 Jul 12 21:47 config.json
-rw-r--r-- 1 root root     456356 Jul 12 21:47 merges.txt
-rw-r--r-- 1 root root 1625537793 Jul 12 21:47 pytorch_model.bin
drwxr-xr-x 3 root root       4096 Jul 12 12:58 runs
-rw-r--r-- 1 root root        239 Jul 12 21:47 special_tokens_map.json
-rw-r--r-- 1 root root    2108689 Jul 12 21:47 tokenizer.json
-rw-r--r-- 1 root root        358 Jul 12 21:47 tokenizer_config.json
-rw-r--r-- 1 root root        196 Jul 12 21:47 train_results.json
-rw-r--r-- 1 root root      19768 Jul 12 21:47 trainer_state.json
-rw-r--r-- 1 root root       3247 Jul 12 21:47 training_args.bin
-rw-r

## Inference

In [11]:
import re
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [12]:
PATH_TO_FINETUNED = '/kaggle/working/tst-summarization'
LOCAL_FILES_ONLY = False
WHITESPACE_HANDLER = lambda k: re.sub('\s+', ' ', re.sub('\n+', ' ', k.strip()))
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model_names = {
    'bart-large-cnn-finetuned-wisoffe': '/kaggle/working/tst-summarization'
              }

In [13]:
 def inference(text, model, tokenizer, out_max_length=27, num_beams=4):
    input_ids = tokenizer(
        [WHITESPACE_HANDLER(text)],
        return_tensors="pt",
        padding="max_length",
        truncation=True,
        max_length=512
    )["input_ids"]

    output_ids = model.generate(
        input_ids=input_ids.to(device),
        max_length=out_max_length, #in tokens
        no_repeat_ngram_size=2,
        num_beams=num_beams
    )[0]

    summary = tokenizer.decode(
        output_ids,
        skip_special_tokens=True,
        clean_up_tokenization_spaces=False
    )
    return summary

In [14]:
train = pd.read_csv("/kaggle/input/title-generation/train.csv")
test = pd.read_csv("/kaggle/input/title-generation/test.csv")
#т.к. в трейн есть дубликаты, необходимо их предварительно удалить
cross = test.merge(train.drop_duplicates(subset=['abstract']), on='abstract', how='left')
cross

,abstract,title
0,Most sequence transformation models use recurr...,NaN
1,The doc2vec approach was introduced as an exte...,NaN
2,LSTM models can vary greatly depending on sequ...,NaN
3,A joint learning process of alignment and tran...,NaN
4,Current unsupervised image-to-image translatio...,NaN
...,...,...
995,subsystem codes are the most versatile class o...,NaN
996,we study dirac-harmonic maps from degenerating...,NaN
997,in this note we study kloosterman sums twisted...,NaN
998,we obtain the rate of growth of long strange s...,"long strange segments, ruin probabilities and ..."


In [15]:
predicted_titles_filename_suffixes = []
OUT_MAX_LENGTH = 20
for model_name in model_names:
    print(model_name)
    predicted_titles_filename_suffixes.append(f"_{model_name}")
    tokenizer = AutoTokenizer.from_pretrained(model_names[model_name], local_files_only=LOCAL_FILES_ONLY)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_names[model_name], local_files_only=LOCAL_FILES_ONLY)
    model.to(device)
    predicted_titles = cross.copy()
    #predicted_titles = predicted_titles.sample(5) #only for test on errors
    predicted_titles
    predicted_titles['title'] = predicted_titles.apply(lambda x: x.title if x.title is not np.NaN 
                                                        else inference(x.abstract, model, tokenizer, out_max_length=OUT_MAX_LENGTH).lower(), 
                                                        axis=1)
    
    # Временный костыль, т.к. generate_csv не принимает названия в одно слово
    predicted_titles['title'] = predicted_titles.title.map(lambda x: x if len(x.split()) > 1 else 'of the and in for a on with to model')
    
    predicted_titles.to_csv(f'predicted_titles{predicted_titles_filename_suffixes[-1]}.csv', index=False)

bart-large-cnn-finetuned-wisoffe


In [16]:
!ls

__notebook__.ipynb				       transformers
predicted_titles_bart-large-cnn-finetuned-wisoffe.csv  tst-summarization
run_train.csv					       wandb


### Делаем submission в Kaggle

In [17]:
import string
from nltk.util import ngrams
import numpy as np
import pandas as pd
import pickle


def generate_csv(input_file='predicted_titles.csv',
                 output_file='submission.csv',
                 voc_file='/kaggle/input/title-generation/vocs.pkl'):
    '''
    Generates file in format required for submitting result to Kaggle
    
    Parameters:
        input_file (str) : path to csv file with your predicted titles.
                           Should have two fields: abstract and title
        output_file (str) : path to output submission file
        voc_file (str) : path to voc.pkl file
    '''
    data = pd.read_csv(input_file)
    with open(voc_file, 'rb') as voc_file:
        vocs = pickle.load(voc_file)

    with open(output_file, 'w') as res_file:
        res_file.write('Id,Predict\n')
        
    output_idx = 0
    for row_idx, row in data.iterrows():
        trg = row['title']
        trg = trg.translate(str.maketrans('', '', string.punctuation)).lower().split()
        trg.extend(['_'.join(ngram) for ngram in list(ngrams(trg, 2)) + list(ngrams(trg, 3))])
        
        VOCAB_stoi = vocs[row_idx]
        trg_intersection = set(VOCAB_stoi.keys()).intersection(set(trg))
        trg_vec = np.zeros(len(VOCAB_stoi))    

        for word in trg_intersection:
            trg_vec[VOCAB_stoi[word]] = 1

        with open(output_file, 'a') as res_file:
            for is_word in trg_vec:
                res_file.write('{0},{1}\n'.format(output_idx, int(is_word)))
                output_idx += 1

In [18]:
for suffix in predicted_titles_filename_suffixes:
    generate_csv(input_file=f'predicted_titles{suffix}.csv',
                 output_file=f'submission{suffix}.csv')

In [19]:
!ls

__notebook__.ipynb				       transformers
predicted_titles_bart-large-cnn-finetuned-wisoffe.csv  tst-summarization
run_train.csv					       wandb
submission_bart-large-cnn-finetuned-wisoffe.csv


In [20]:
!rm transformers

rm: cannot remove 'transformers': Is a directory
